In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


In [2]:
def reduce_memory(df):
    int_cols = list(df.select_dtypes(include=['int']).columns)
    for col in int_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)
    gc.collect()


In [3]:
##################################################
# Load user_logs 
##################################################
user_logs = pd.read_csv('../../input/user_logs_v2.csv')
#user_logs.sort_values(by = ['msno', 'date'],inplace = True)

In [4]:
user_logs = user_logs.head(5000)

In [6]:
reduce_memory(user_logs)
user_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
msno          5000 non-null object
date          5000 non-null int32
num_25        5000 non-null int16
num_50        5000 non-null int8
num_75        5000 non-null int8
num_985       5000 non-null int16
num_100       5000 non-null int16
num_unq       5000 non-null int16
total_secs    5000 non-null float32
dtypes: float32(1), int16(4), int32(1), int8(2), object(1)
memory usage: 127.0+ KB


In [5]:
user_logs = user_logs.head(n = 5000)

In [17]:
user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
7571198,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170301,1,0,0,1,-117,-117,31748
8050276,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170302,2,0,0,0,12,12,2882
12682180,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170304,7,2,1,0,9,16,2582
16814224,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170305,0,0,0,0,1,1,230
12318951,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,20170307,1,0,1,1,9,11,2644


# 此消費者聽歌佔一整天的比例-->把它分區間,判斷他是重度患者(>0.5),中度患者(), 低度患者(< 0.125)

In [22]:
def habit_discrimination(x):
    # 0.5: 一天聽12小時
    # 0.333: 一天聽8小時
    # 0.125: 一天聽3小時
    # 0.0416: 一天聽1小時
    if x > 0.5:
        return 1 # 重度kkbox依賴者
    elif 0.5 > x > 0.125:
        return 2
    elif 0.125 > x > 0.0416:
        return 3
    else:
        return 4
user_logs['total_secs_percentage'] = user_logs.total_secs.apply(lambda x: x / (24*60*60))
user_logs['listening_habit_zone'] = user_logs.total_secs_percentage.apply(habit_discrimination)

# 此消費者聽歌已累積多少時間 (Number of time user has logged on)

In [48]:
def make_order_number(x):
    x['order_number'] = [i+1 for i in range(x.shape[0])]
    return x
user_logs['num_of_time_the_user_has_logged_in'] = user_logs.groupby('msno').total_secs.cumsum()
user_logs = user_logs.groupby('msno').apply(make_order_number)
user_logs['num_of_time_the_user_has_logged_in_ratio'] = user_logs.num_of_time_the_user_has_logged_in / user_logs.order_number

# 多少完整的歌和多少不完整的歌 (Number of completed songs against incomplete songs)
# completed songs = num_100 + num_985
# incompleted songs = num_75 + num_50 + num_25

In [26]:
def completed_vs_incompleted_songs(x):
    x['num_completed_songs'] = x.num_100 + x.num_985
    x['num_incompleted_songs'] = x.num_25 + x.num_50 + x.num_75
    return x
user_logs = user_logs.apply(completed_vs_incompleted_songs, axis = 1)

AttributeError: ("'Series' object has no attribute 'num_100'", 'occurred at index 0')

In [37]:
user_logs['completed_songs_ratio'] = user_logs.num_completed_songs/ (user_logs.num_incompleted_songs + user_logs.num_completed_songs)

In [42]:
user_logs['is_satisfied'] = user_logs.completed_songs_ratio.apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
# user_logs.drop(['total_secs'], axis = 1,inplace = True)

# num_100佔所有聽過的歌曲中的比例, 這個比例越高代表這天他們很穩定聽歌!
# num_985佔所有聽過的歌曲中的比例
# num_75佔所有聽過的歌曲中的比例
# num_50佔所有聽過的歌曲中的比例
# num_25佔所有聽過的歌曲中的比例, 這個比例越高代表這天他們一直在找歌!



In [7]:
user_logs = user_logs.head(n = 5000)

In [6]:
# # it's extremly time-consuming
# def get_ratio(x):
#     x['num_100_ratio'] = x.num_100/ (x.num_25 + x.num_50 + x.num_75 + x.num_985 + x.num_100)
#     x['num_985_ratio'] = x.num_985/ (x.num_25 + x.num_50 + x.num_75 + x.num_985 + x.num_100)
#     x['num_75_ratio'] = x.num_75/ (x.num_25 + x.num_50 + x.num_75 + x.num_985 + x.num_100)
#     x['num_50_ratio'] = x.num_50/ (x.num_25 + x.num_50 + x.num_75 + x.num_985 + x.num_100)
#     x['num_25_ratio'] = x.num_25/ (x.num_25 + x.num_50 + x.num_75 + x.num_985 + x.num_100)
#     return x
# user_logs = user_logs.apply(get_ratio, axis = 1)

In [8]:
user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.272949
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.698975
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337891
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454102
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558594


In [9]:
user_logs.loc[:,"num_25":"num_100"] = user_logs.loc[:,"num_25":"num_100"].div(user_logs.loc[:,"num_25":"num_100"].sum(axis=1), axis=0)


In [23]:
user_logs.rename(columns = {'num_25':'num_25_ratio','num_50':'num_50_ratio',
                           'num_75':'num_75_ratio','num_985':'num_985_ratio',
                           'num_100':'num_100_ratio'}, inplace =True)

In [25]:
user_logs.head()

,msno,date,num_25_ratio,num_50_ratio,num_75_ratio,num_985_ratio,num_100_ratio,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,0.235294,0.117647,0.000000,0.029412,0.617647,18,6309.272949
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,0.142857,0.142857,0.071429,0.000000,0.642857,11,2390.698975
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,0.353741,0.020408,0.034014,0.020408,0.571429,110,23203.337891
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,0.866995,0.019704,0.009852,0.009852,0.093596,191,7100.454102
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,0.017241,0.008621,0.000000,0.008621,0.965517,93,28401.558594


In [31]:
col = ['msno','date','num_25','num_50','num_75','num_985','num_100']
df = user_logs[col]
#df.div(df.sum(axis=1), axis=0)
df.loc[:,"num_25":"num_100"] = df.loc[:,"num_25":"num_100"].div(df.loc[:,"num_25":"num_100"].sum(axis=1), axis=0)
df

,msno,date,num_25,num_50,num_75,num_985,num_100
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,0.235294,0.117647,0.000000,0.029412,0.617647
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,0.142857,0.142857,0.071429,0.000000,0.642857
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,0.353741,0.020408,0.034014,0.020408,0.571429
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,0.866995,0.019704,0.009852,0.009852,0.093596
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,0.017241,0.008621,0.000000,0.008621,0.965517
5,qR/ndQ5B+1cY+c9ihwLoiz+RFiqEnGyQKo32ZErEVKo=,20170331,0.071429,0.000000,0.000000,0.000000,0.928571
6,N6ch5ArfJixq9mvARJv9ip1D11uvDVazz6Qup5mX7DI=,20170330,0.321429,0.035714,0.000000,0.000000,0.642857
7,JEjl2W1ivEI6epeobW3n5rDg/r6PJm4OJhtmhwYaQIc=,20170331,0.581994,0.218650,0.016077,0.009646,0.173633
8,lPK4IYIFdfTT6pq7x2dSV1Vy9j9hNMWzf7XfYBUBz5E=,20170331,0.016129,0.000000,0.005376,0.005376,0.973118
9,cDgnvwQwCwru2m3zcSuK5oTr6pr1nMPQ/6RerOmxrs0=,20170331,0.121951,0.097561,0.024390,0.024390,0.731707


# number of repeated songs

In [15]:
def num_repeated_songs(x):
    x['num_repeated_songs'] = (x.num_100 + x.num_985 + x.num_75) / x.num_unq
    return x
user_logs = user_logs.apply(num_repeated_songs, axis = 1)

# Personal Analysis

In [32]:
df = user_logs[user_logs.msno == '+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=']

# 此消費者是連續每天都聽麼, 還是隔幾天聽 (Duration of Membership) [時間差的feature]

In [33]:
df

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs


# range of date

In [15]:
user_logs.sort_values(by = 'date').date.unique()

array([20170301, 20170302, 20170303, 20170304, 20170305, 20170306,
       20170307, 20170308, 20170309, 20170310, 20170311, 20170312,
       20170313, 20170314, 20170315, 20170316, 20170317, 20170318,
       20170319, 20170320, 20170321, 20170322, 20170323, 20170324,
       20170325, 20170326, 20170327, 20170328, 20170329, 20170330, 20170331])

In [8]:
test = pd.read_csv('../input/sample_submission_v2.csv')

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907471 entries, 0 to 907470
Data columns (total 2 columns):
msno        907471 non-null object
is_churn    907471 non-null int64
dtypes: int64(1), object(1)
memory usage: 13.8+ MB


# test中哪些user不再user_logs_v2中:總共有207242

In [15]:
test[~test.msno.isin(user_logs.msno)].shape # test中哪些user不再user_logs_v2中:總共有207242


(207242, 2)

In [13]:
train = pd.read_csv('../input/train_v2.csv')

In [31]:
type(train.as_matrix()) == type(train.values)

True

# train中哪些user再user_logs_v2中:總共有754551

In [16]:
train[train.msno.isin(user_logs.msno)].shape

(754551, 2)

# The same user may appear in different output files.

In [ ]:
##################################################
# Load user_logs 
##################################################
user_logs = pd.read_csv('../input/user_logs.csv') # 392,106,544 rows --> nealy 400 millions rows

# splitting user_logs

In [21]:
import sklearn 
import matplotlib
import matplotlib.pyplot as plt



In [2]:
def load_user_logs():
    # 切割user_logs
    print("Loading training Data")
    train = pd.read_csv("../../input/train_v2.csv")
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    utils.reduce_memory(train)
    print("Loading test Data")
    test = pd.read_csv("../../input/sample_submission_v2.csv")
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    utils.reduce_memory(test)

    valid_msno = pd.concat([train, test])['msno'].as_matrix() # DataFrame.as_matrix()  == DataFrame.values
    train = []
    test = []
    # chunksize=10,000,000
    user_logs_iter = pd.read_csv("../../input/user_logs_v2.csv", chunksize=1000000, 
                iterator=True, low_memory=False, parse_dates=['date']) # return a generator
    user_logs = pd.DataFrame()
    user_log_counts = pd.DataFrame() # for append DataFrame

    i = 0
    for df in user_logs_iter:
        df = df[df['msno'].isin(valid_msno)] # 找出chunked_user_logs中msno有在valid_msno的dataframe
        #==============================================================================
        print('reduce memory')
        #==============================================================================
        utils.reduce_memory(df)
        if i is 0: 
            user_logs = df
            user_log_counts = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
            user_log_counts.columns = ['msno','logs_count']
        else:
            temp_user_log_counts = pd.DataFrame(df['msno'].value_counts().reset_index())
            temp_user_log_counts.columns = ['msno','logs_count']
            user_logs = pd.concat([user_logs, df])
            user_log_counts = pd.concat([user_log_counts, temp_user_log_counts])
        
        user_logs = user_logs.groupby(["msno"], as_index=False)["num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs"].sum()        
        user_log_counts = user_log_counts.groupby(["msno"], as_index=False)["logs_count"].sum()        

        print("User Logs {} df: {}".format(str(i), user_logs.shape))
        print("User Log Counts {} df: {}".format(str(i), user_log_counts.shape))     
        i = i+1
            
    print(user_logs.shape)
    user_logs = pd.merge(user_logs, user_log_counts, how='left', on='msno')
    print(user_logs.shape)
    
    return user_logs

## Suggest commenting out all logs except for the user_logs when fetching user logs
## and vice versa
def raw_daq():
    print("Loading user Log Data")
    user_logs = load_user_logs() # call
    print("User Logs set shape {}".format(user_logs.shape))
    # write
    path = '../../input/preprocessed_data/compressed_user_logs'
    gc.collect()
    utils.to_multiple_csv(user_logs, path, split_size = 40)

In [3]:
print("Getting raw data")
raw_daq()

Getting raw data
Loading user Log Data
Loading training Data


100%|██████████| 2/2 [00:00<00:00, 1737.49it/s]

reduce memory
Loading test Data



100%|██████████| 2/2 [00:00<00:00, 1757.51it/s]

reduce memory


...

reduce memory


User Logs 18 df: (842708, 8)
User Log Counts 18 df: (842708, 2)
(842708, 8)


  0%|          | 0/40 [00:00<?, ?it/s]

(842708, 9)
User Logs set shape (842708, 9)


100%|██████████| 40/40 [00:06<00:00,  5.75it/s]


In [11]:
# 切割user_logs
print("Loading training Data")
train = pd.read_csv("../../input/train_v2.csv")
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(train)
print("Loading test Data")
test = pd.read_csv("../../input/sample_submission_v2.csv")
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(test)

valid_msno = pd.concat([train, test])['msno'].as_matrix() # DataFrame.as_matrix()  == DataFrame.values


Loading training Data


100%|██████████| 2/2 [00:00<00:00, 2444.95it/s]

reduce memory
Loading test Data



100%|██████████| 2/2 [00:00<00:00, 2513.06it/s]

reduce memory


In [37]:
valid_msno.shape

(1878431,)

In [7]:
user_logs_iter = pd.read_csv("../../input/user_logs_v2.csv", chunksize=50000, 
                iterator=True, low_memory=False, parse_dates=['date'])


In [45]:
user_logs = pd.DataFrame()
user_log_counts = pd.DataFrame()

i = 0
for df in user_logs_iter:
    df = df[df['msno'].isin(valid_msno)] #找出chunked_user_logs中msno有在valid_msno的dataframe
    #==============================================================================
    print('reduce memory')
    #==============================================================================
    utils.reduce_memory(df)
    if i is 0: 
        user_logs = df
        user_log_counts = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
        user_log_counts.columns = ['msno','logs_count']
    else:
        temp_user_log_counts = pd.DataFrame(df['msno'].value_counts().reset_index())
        temp_user_log_counts.columns = ['msno','logs_count']
        user_logs = pd.concat([user_logs, df])
        user_log_counts = pd.concat([user_log_counts, temp_user_log_counts])

    user_logs = user_logs.groupby(["msno"], as_index=False)["num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs"].sum()        
    user_log_counts = user_log_counts.groupby(["msno"], as_index=False)["logs_count"].sum()        
    # 
    print (user_logs.columns)
    print (user_log_counts.columns)
    print("User Logs {} df: {}".format(str(i), user_logs.shape))
    print("User Log Counts {} df: {}".format(str(i), user_log_counts.shape))     
    i = i+1
    if i > 5:
        break
# print(user_logs.shape)
# user_logs = pd.merge(user_logs, user_log_counts, how='left', on='msno')
# print(user_logs.shape)


100%|██████████| 9/9 [00:00<00:00, 1161.18it/s]

reduce memory


Index(['msno', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')
Index(['msno', 'logs_count'], dtype='object')
User Logs 0 df: (39354, 8)
User Log Counts 0 df: (39354, 2)


100%|██████████| 9/9 [00:00<00:00, 1171.74it/s]

reduce memory


...

reduce memory


Index(['msno', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')
Index(['msno', 'logs_count'], dtype='object')
User Logs 4 df: (175301, 8)
User Log Counts 4 df: (175301, 2)


100%|██████████| 9/9 [00:00<00:00, 1378.70it/s]

reduce memory


Index(['msno', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')
Index(['msno', 'logs_count'], dtype='object')
User Logs 5 df: (204473, 8)
User Log Counts 5 df: (204473, 2)


In [26]:
user_logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
3750000,iNfXK2nICSuVxPbr0bcOKaI2N0AiAcqu0sLf8dKsweM=,2017-03-16,12,0,0,1,9,14,2370.635010
3750001,44FGuOUu81dmVySYYIZ0WnKI2/tatjLGmUAuMOGqRO0=,2017-03-15,0,0,1,0,8,8,1311.464966
3750002,OFIwwwQCT+VI3ZSOpHqh1ZFo0reAzWukpPkF8prwNns=,2017-03-13,3,0,0,0,19,22,4086.444092
3750003,B5NY2FogzQ09+E0Tl+IKb/l7+0FXAMftBLc41IYtn2E=,2017-03-14,0,1,0,0,5,1,1579.634033
3750004,u2Ss6qVNaKouQTukxNlCh8EuF39g4BqBnQAOeEPEDXY=,2017-03-25,1,0,0,0,5,6,1123.996948


In [41]:
user_logs[user_logs.msno == '29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=']

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
3768255,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,2017-03-02,24,4,2,13,39,24,12968.190430
3772269,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,2017-03-19,12,3,5,12,49,43,15661.371094
3782573,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,2017-03-30,2,1,1,6,23,19,7025.250977


In [27]:
user_log_counts.head()

,msno,logs_count
0,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,3
1,WQ1IOc6fnUMOFvN7wL2ec38KhyRUiASrgFlOcxg2oXM=,3
2,T7bE8mgakAqmTiwFnDqsAe9IdsLF59Ibx88J0rDdODU=,3
3,v76ZgdPjg6MluzDEaje2dbhI1v1leZq5gu0B5wCiOfE=,3
4,PMNmR1RE3b4roz/PW5VqVQ7tY9NU/SXaHrSDPYirh8U=,3


In [39]:
col = ["num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs"] # remove date# ignore time dimension
d = user_logs.groupby(["msno"], as_index=False)[col].sum()    
d[d.msno == '29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=']

,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
4996,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,38,8,8.0,31,111,86,35654.8125


In [38]:
e = user_log_counts.groupby(["msno"], as_index=False)["logs_count"].sum()
e[e.msno == '29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=']

,msno,logs_count
4996,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,3


In [44]:
f = pd.merge(d, e, how='left', on='msno')
f[f.msno == '29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=']

,msno,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,logs_count
4996,29V0Jm3Xli1dy9UFeEL/BH2EMOr62DgeGLeKAKfE07k=,38,8,8.0,31,111,86,35654.8125,3


In [4]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


##################################################
# Load user_logs
##################################################
# chunksize=10,000,000 (10 millions)
user_logs_iter = pd.read_csv("../../input/user_logs.csv", chunksize=10000, 
            iterator=True, low_memory=False, parse_dates=['date'])

#==============================================================================
#def
#==============================================================================

def make(T):
    if T == -1:
        folder = 'test'
        test = pd.read_csv('../../input/sample_submission_v2.csv')
        #user_logs_march = pd.read_csv('../input/user_logs_v2.csv')
        #==============================================================================
        print('reduce memory')
        #==============================================================================
        utils.reduce_memory(test)
        #utils.reduce_memory(user_logs_march)
        
        valid_msno = test['msno'].as_matrix()
    else:
        col = ['msno']
        folder = 'trainW-'+str(T)
        train = pd.read_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T))[col]     
        #==============================================================================
        print('reduce memory')
        #==============================================================================
        utils.reduce_memory(train)
        
        valid_msno = train['msno'].as_matrix()  
    ##################################################
    #core
    ##################################################
    c = 0
    chunked_user_logs = pd.DataFrame() 
    start_time = time.time()

    for df in user_logs_iter:
        c += 1
        #==============================================================================
        print('reduce memory')
        #==============================================================================
        utils.reduce_memory(df)

        df = df[df['msno'].isin(valid_msno)] #找出chunked_user_logs中msno有在valid_msno的dataframe      
        df.sort_values(by = ['msno','date']).reset_index(drop = True)
        gc.collect()
        ##################################################
        # Convert string to datetime format
        ##################################################
        # 這邊每一個loop花5分鐘的話
        #print ('befor',df)
        df['date']  = df.date.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
        #print ('after',df)
        ##################################################
        # Filtering accroding to w
        ##################################################
        if T == 0:
            # w = 0:使用3月之前的資料當作history
            df = df[ (df.date < datetime.strptime('2017-03-01', '%Y-%m-%d'))]
        elif T == 1:
            # w = 1:使用2月之前的資料當作history
            df = df[ (df.date < datetime.strptime('2017-02-01', '%Y-%m-%d'))]
        elif T == 2:
            # w = 2:使用1月之前的資料當作history
            df = df[ (df.date < datetime.strptime('2017-01-01', '%Y-%m-%d'))]
        elif T == -1:
            # w = -1:使用4月之前的資料當作history
            df = df[ (df.date < datetime.strptime('2017-04-01', '%Y-%m-%d'))]
        chunked_user_logs = chunked_user_logs.append(df)
        print("Loop ",c,"took %s seconds" % (time.time() - start_time))
        #print("User Logs {} df: {}".format(str(c), df.shape))
        if c > 5:
            break
    print("Shape of Chunked User Logs: {}".format(chunked_user_logs.shape))
    ##################################################
    # write
    ##################################################
    path = '../feature/{}/compressed_user_logs'.format(folder)
    gc.collect()
    utils.to_multiple_csv(chunked_user_logs,path, split_size = 40)



In [5]:
##################################################
# Main
##################################################
s = time.time()
make(-1)
e = time.time()
print (e-s)


100%|██████████| 9/9 [00:00<00:00, 771.26it/s]

reduce memory
reduce memory



100%|██████████| 9/9 [00:00<00:00, 541.81it/s]

Loop  1 took 0.884397029876709 seconds
reduce memory



100%|██████████| 9/9 [00:00<00:00, 947.25it/s]

...

Loop  5 took 4.239886045455933 seconds
reduce memory



  0%|          | 0/40 [00:00<?, ?it/s]

Loop  6 took 5.08499813079834 seconds
Shape of Chunked User Logs: (30067, 9)


100%|██████████| 40/40 [00:02<00:00, 14.41it/s]

9.612336158752441


In [64]:
T = 0
folder = 'trainW-'+str(T)
user_logs = utils.read_multiple_csv('../../feature/{}/compressed_user_logs'.format(folder))


100%|██████████| 40/40 [05:19<00:00,  7.99s/it]


In [66]:
user_logs[user_logs.num_25 <0]

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
5610836,hAuFB/XFISrQWRqm1bl30H3LM5my0fZs30S6BMB94Yc=,2015-08-29,-100,57,66,24,23,277,2.696392e+04
5611355,kp33qJsM9w/oH5+euYXOPHxxuNoVYs7TsBcEnQ6I3NM=,2015-10-09,-65,11,7,12,37,212,1.218050e+04
5612140,I9i01ZHvrJzNtZPdpwAikxVL48+hyNemZC31F4fTm2U=,2016-07-08,-55,6,0,1,0,193,2.758248e+03
5613106,UDQmx2h+l5QZcZ3Dsjk1EajXQSpR78o97vKzqnjYxYo=,2016-04-15,-123,5,5,6,84,219,2.388755e+04
5613212,rrrrCTU1y+LAD/Kv/LJpwi3t7qm74NrRSyELTUPRrt4=,2015-05-30,-125,7,0,3,78,179,1.922832e+04
5613660,HNEUQHGdMI090Xo3h2f/RhB9y09TsqxHIEuE/rhUYqU=,2016-03-05,-123,2,1,4,35,168,9.710256e+03
5616100,RJdd69KEPtNtyWXR4p0GJNH4WuEftf4awSwZmtZ8bVI=,2016-11-04,-75,14,5,13,53,177,1.922162e+04
5617558,9bfAfYAtY6EAel020S1UtgalxdR7nEE+xeTWknEsDOo=,2016-10-12,-110,7,12,1,38,189,1.407540e+04
5617638,m0IMiU7cB8GG2tE/io8YZZvKmVa9tSxiDnSoqFP6OSo=,2016-10-09,-27,15,8,3,53,169,1.799832e+04
5617901,5cd/yVuvXN0xAPE5rBsv083OU41HNHSFy4LrzYMobgs=,2016-12-27,-81,64,19,5,159,382,5.235464e+04


In [ ]:
user_logs[user_logs.num_25 <0]